In [2]:
import torch
import numpy as np

# Parameters
num_nodes = 5  # Number of sectors
seq_len = 12  # Sequence length (e.g., months)
batch_size = 32  # Batch size

# Dummy data for sectorial export trade volumes
np.random.seed(0)  # For reproducibility
X = np.random.rand(batch_size, num_nodes, seq_len)  # Input data
X = torch.from_numpy(X).float()  # Convert to PyTorch tensor

# Dummy adjacency matrix (assuming sectors are interconnected)
A = torch.ones(num_nodes, num_nodes)  # Simple adjacency matrix

# Dummy static features (if applicable)
FE = None  # No static features in this example


In [10]:
from torch_geometric_temporal.nn.attention.mtgnn import MTGNNLayer, GraphConstructor
# Parameters for MTGNNLayer
dilation_exponential = 2
rf_size_i = 4
kernel_size = 1
j = 1
residual_channels = 32
conv_channels = 32
skip_channels = 64
kernel_set = [2, 4, 8]
new_dilation = 1
layer_norm_affline = True
gcn_true = True
seq_length = seq_len
receptive_field = 8
dropout = 0.1
gcn_depth = 2
num_nodes = num_nodes
propalpha = 0.5

# Initialize MTGNN layer
mtgnn_layer = MTGNNLayer(
    dilation_exponential,
    rf_size_i,
    kernel_size,
    j,
    residual_channels,
    conv_channels,
    skip_channels,
    kernel_set,
    new_dilation,
    layer_norm_affline,
    gcn_true,
    seq_length,
    receptive_field,
    dropout,
    gcn_depth,
    num_nodes,
    propalpha,
)



graph_constructor = GraphConstructor(
    nnodes=num_nodes, k=3, dim=16, alpha=0.5, xd=None
)

# Construct adjacency matrix using graph constructor (optional)
A_tilde = graph_constructor(torch.arange(num_nodes))

In [9]:
print(X.shape)

torch.Size([32, 5, 12])


In [7]:
torch.arange(num_nodes)

tensor([0, 1, 2, 3, 4])

In [11]:
X_skip = X

# Perform forward pass
output = mtgnn_layer(X, X_skip, A, torch.arange(num_nodes), training=True)

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(mtgnn_layer.parameters(), lr=0.001)

# Training loop
for epoch in range(100):  # Example number of epochs
    optimizer.zero_grad()
    
    # Forward pass
    output = mtgnn_layer(X, X_skip, A, torch.arange(num_nodes), training=True)
    
    # Calculate loss (assuming target is similar to output for demonstration)
    target = output.detach()  # For demonstration; use actual target data
    loss = criterion(output, target)
    
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional
from torch_geometric_temporal.nn.attention.mtgnn import MTGNNLayer

# ------------------------------------------------------------------------------
# Assume that the following components have been implemented and imported:
# - DilatedInception
# - MixProp
# - LayerNormalization
# - MTGNNLayer (as shown in your snippet)
# ------------------------------------------------------------------------------
# For demonstration purposes, we assume these components exist in your library.

class SectorialTradePredictor(nn.Module):
    def __init__(
        self,
        num_nodes: int,
        in_channels: int,
        out_channels: int,
        seq_length: int,
        forecast_horizon: int,
        num_layers: int = 3,
        residual_channels: int = 32,
        conv_channels: int = 32,
        skip_channels: int = 64,
        kernel_size: int = 2,
        dilation_exponential: int = 2,
        kernel_set: list = [2, 3, 6, 7],
        layer_norm_affline: bool = True,
        gcn_true: bool = True,
        gcn_depth: int = 2,
        dropout: float = 0.3,
        propalpha: float = 0.05,
    ):
        """
        Args:
            num_nodes (int): Number of countries (graph nodes).
            in_channels (int): Number of input channels (e.g., sectors).
            out_channels (int): Number of output channels (typically same as in_channels).
            seq_length (int): Length of historical time series.
            forecast_horizon (int): Number of future time steps to predict.
            num_layers (int): Number of stacked MTGNN layers.
            residual_channels (int): Channels for residual mapping.
            conv_channels (int): Channels for the dilated convolutions.
            skip_channels (int): Channels for skip connections.
            kernel_size (int): Kernel size for dilated convolutions.
            dilation_exponential (int): Dilation factor base.
            kernel_set (list): List of kernel sizes for the DilatedInception block.
            layer_norm_affline (bool): Whether layer norm is affine.
            gcn_true (bool): Whether to include graph convolution in the MTGNNLayer.
            gcn_depth (int): Depth for the graph convolution.
            dropout (float): Dropout rate.
            propalpha (float): MixHop propagation parameter.
        """
        super(SectorialTradePredictor, self).__init__()
        self.num_layers = num_layers
        self.num_nodes = num_nodes
        self.forecast_horizon = forecast_horizon

        # Calculate overall receptive field (this simple calculation may need adjustment)
        self.receptive_field = 1
        for i in range(num_layers):
            dilation = dilation_exponential ** i
            self.receptive_field += (kernel_size - 1) * dilation

        # Initial convolution to lift input dimension to residual_channels
        self.input_conv = nn.Conv2d(in_channels, residual_channels, kernel_size=(1, 1))
        
        # Build a stack of MTGNN layers
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            dilation = dilation_exponential ** i
            layer = MTGNNLayer(
                dilation_exponential=dilation_exponential,
                rf_size_i=self.receptive_field,  # using the full receptive field
                kernel_size=kernel_size,
                j=i,
                residual_channels=residual_channels,
                conv_channels=conv_channels,
                skip_channels=skip_channels,
                kernel_set=kernel_set,
                new_dilation=dilation,
                layer_norm_affline=layer_norm_affline,
                gcn_true=gcn_true,
                seq_length=seq_length,
                receptive_field=self.receptive_field,
                dropout=dropout,
                gcn_depth=gcn_depth,
                num_nodes=num_nodes,
                propalpha=propalpha,
            )
            self.layers.append(layer)
        
        # Final convolutions to produce the prediction.
        # The first one processes the aggregated skip connection;
        # the second one maps to (forecast_horizon * out_channels).
        self.end_conv_1 = nn.Conv2d(skip_channels, skip_channels, kernel_size=(1, 1))
        self.end_conv_2 = nn.Conv2d(skip_channels, forecast_horizon * out_channels, kernel_size=(1, 1))
    
    def forward(self, X: torch.FloatTensor, A_tilde: Optional[torch.FloatTensor], training: bool = True):
        """
        Args:
            X (torch.FloatTensor): Input tensor with shape (batch_size, in_channels, num_nodes, seq_length)
            A_tilde (torch.FloatTensor or None): Predefined adjacency matrix (num_nodes, num_nodes) with trade relationship scores.
            training (bool): Whether the model is in training mode.
        
        Returns:
            torch.FloatTensor: Predictions with shape (batch_size, out_channels, num_nodes, forecast_horizon)
        """
        # Initial projection to match residual channels
        X = self.input_conv(X)
        batch_size, _, num_nodes, seq_len = X.shape
        
        # Initialize the skip connection tensor.
        # We use the out_channels of the skip_conv from the first layer.
        skip_channels = self.layers[0]._skip_conv.out_channels
        X_skip = torch.zeros(batch_size, skip_channels, num_nodes, seq_len, device=X.device)
        
        # Pass through each MTGNN layer
        for layer in self.layers:
            # Create an index tensor for layer normalization (based on current sequence length)
            current_seq_len = X.size(3)
            idx = torch.arange(current_seq_len).unsqueeze(0).expand(batch_size, -1).to(X.device)
            X, X_skip = layer(X, X_skip, A_tilde, idx, training)
        
        # Final convolutions on the aggregated skip connection to obtain predictions.
        Y = F.relu(X_skip)
        Y = F.relu(self.end_conv_1(Y))
        Y = self.end_conv_2(Y)
        # Assuming Y has shape (batch_size, forecast_horizon*out_channels, num_nodes, 1)
        # We squeeze the last dimension and then reshape to (batch_size, out_channels, num_nodes, forecast_horizon)
        Y = Y.squeeze(-1)
        Y = Y.view(batch_size, self.forecast_horizon, -1, num_nodes)
        # Optionally, permute dimensions to (batch_size, out_channels, num_nodes, forecast_horizon)
        Y = Y.permute(0, 2, 3, 1)
        return Y

# ------------------------------------------------------------------------------
# Example usage:
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Example parameters:
    num_nodes = 10         # e.g., 10 countries
    in_channels = 5        # e.g., 5 sectors (input features)
    out_channels = 5       # predict the same number of sectors
    seq_length = 24        # using the last 24 time steps (e.g., months)
    forecast_horizon = 3   # forecast the next 3 time steps
    batch_size = 16

    # Instantiate the predictor
    model = SectorialTradePredictor(
        num_nodes=num_nodes,
        in_channels=in_channels,
        out_channels=out_channels,
        seq_length=seq_length,
        forecast_horizon=forecast_horizon
    )
    
    # Dummy input tensor: (batch_size, in_channels, num_nodes, seq_length)
    X = torch.randn(batch_size, in_channels, num_nodes, seq_length)
    # Dummy adjacency matrix (A_tilde): (num_nodes, num_nodes)
    A_tilde = torch.rand(num_nodes, num_nodes)
    
    # Run the model
    predictions = model(X, A_tilde, training=True)
    print("Predictions shape:", predictions.shape)

RuntimeError: The size of tensor a (2) must match the size of tensor b (24) at non-singleton dimension 3

In [13]:
class SectorTradeForecaster(nn.Module):
    def __init__(self, num_sectors, num_countries, seq_len, adj_matrix):
        super().__init__()
        self.num_sectors = num_sectors
        self.num_countries = num_countries
        self.seq_len = seq_len
        self.register_buffer('A_tilde', adj_matrix)  # Trade relationship matrix

        # MTGNN Layer Parameters
        self.mtgnn = MTGNNLayer(
            dilation_exponential=1,
            rf_size_i=1,
            kernel_size=3,
            j=2,  # Adjust based on desired receptive field
            residual_channels=num_sectors,
            conv_channels=64,
            skip_channels=64,
            kernel_set=[2, 3, 6, 7],
            new_dilation=1,
            layer_norm_affline=True,
            gcn_true=True,
            seq_length=seq_len,
            receptive_field=5,  # Match input sequence after j=2
            dropout=0.2,
            gcn_depth=2,
            num_nodes=num_countries,
            propalpha=0.05
        )

        # Final prediction layer
        self.output_conv = nn.Conv2d(64, num_sectors, kernel_size=(1, 1))

    def forward(self, x):
        X_skip = 0  # Initialize skip connection
        _, X_skip = self.mtgnn(x, X_skip, self.A_tilde, None, self.training)
        return self.output_conv(X_skip).squeeze(-1)  # (batch, sectors, countries)

In [16]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

class TradeVolumeDataset(Dataset):
    def __init__(self, data, seq_length, pred_length=1, split='train', split_ratio=0.8):
        """
        Args:
            data: Input tensor of shape (num_countries, num_sectors, total_timesteps)
            seq_length: Length of input sequence
            pred_length: Length of prediction sequence
            split: 'train' or 'val'
            split_ratio: Ratio for train-val split
        """
        # Convert to (num_sectors, num_countries, timesteps) for MTGNN compatibility
        self.data = data.permute(1, 0, 2)
        self.seq_length = seq_length
        self.pred_length = pred_length
        self.split = split
        
        # Normalize data
        self.scaler = StandardScaler()
        scaled_data = self.scaler.fit_transform(self.data.reshape(-1, self.data.shape[-1]))
        self.data = torch.tensor(scaled_data.reshape(self.data.shape), dtype=torch.float32)
        
        # Split data
        total_timesteps = self.data.shape[-1]
        split_idx = int(total_timesteps * split_ratio)
        
        if split == 'train':
            self.data = self.data[..., :split_idx]
        else:
            self.data = self.data[..., split_idx:]

    def __len__(self):
        return self.data.shape[-1] - self.seq_length - self.pred_length + 1

    def __getitem__(self, idx):
        # Get input sequence
        start = idx
        end = start + self.seq_length
        x = self.data[..., start:end]
        
        # Get target (next time step for all sectors and countries)
        y = self.data[..., end:end+self.pred_length].squeeze(-1)
        
        return x, y

def create_dataloaders(data, seq_length, batch_size=32):
    """
    Create train and validation dataloaders
    
    Args:
        data: Raw input tensor (num_countries, num_sectors, total_timesteps)
        seq_length: Input sequence length
        batch_size: Batch size
    
    Returns:
        train_loader, val_loader
    """
    train_dataset = TradeVolumeDataset(
        data=data,
        seq_length=seq_length,
        split='train'
    )
    
    val_dataset = TradeVolumeDataset(
        data=data,
        seq_length=seq_length,
        split='val'
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        drop_last=True
    )
    
    return train_loader, val_loader

# Example usage -------------------------------------------------
if __name__ == "__main__":
    # Generate synthetic data
    num_countries = 30
    num_sectors = 5
    total_timesteps = 100
    seq_length = 12
    
    # Synthetic data: (countries, sectors, timesteps)
    raw_data = torch.rand(num_countries, num_sectors, total_timesteps)
    
    # Create dataloaders
    train_loader, val_loader = create_dataloaders(
        data=raw_data,
        seq_length=seq_length,
        batch_size=32
    )
    
    # Verify batch shape
    sample_x, sample_y = next(iter(train_loader))
    print(f"Input shape: {sample_x.shape}")  # Should be (batch, sectors, countries, seq_length)
    print(f"Target shape: {sample_y.shape}")  # Should be (batch, sectors, countries)

Input shape: torch.Size([32, 5, 30, 12])
Target shape: torch.Size([32, 5, 30])


In [17]:
# Example parameters
adj_matrix = torch.rand(num_countries, num_countries)  # Replace with real data

model = SectorTradeForecaster(num_sectors, num_countries, seq_len, adj_matrix)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    for x_batch, y_batch in train_loader:  # y_batch: (batch, sectors, countries)
        optimizer.zero_grad()
        output = model(x_batch)  # Predicted next time step
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()



RuntimeError: Calculated padded input size per channel: (30 x 6). Kernel size: (1 x 8). Kernel size can't be greater than actual input size